In [ ]:
import pandas as pd

In [ ]:
# importing the train and test data dataset from file
train_val_csv = pd.read_csv('UNSW_NB15_training-set.csv')
test_csv = pd.read_csv('UNSW_NB15_testing-set.csv')

In [ ]:
train_val_csv=train_val_csv.drop(columns=['id', 'proto', 'service', 'state'])
test_csv=test_csv.drop(columns=['id', 'proto', 'service', 'state'])

# train_val_csv=train_val_csv.drop(columns=['id'])
# test_csv=test_csv.drop(columns=['id'])

In [ ]:
from sklearn.model_selection import train_test_split

In [ ]:
train,val = train_test_split(train_val_csv, test_size=0.2, random_state = 1)
train_val_csv = train

In [ ]:
print(len(train), 'training examples')
print(len(val), 'validation examples')
print(len(test_csv), 'test examples')

In [ ]:
train_val_csv.head(n=5)

In [ ]:
train_val_csv.info()

In [ ]:
train_val_csv.isnull().sum()

In [ ]:
train_val_csv.shape

In [ ]:

data_to_use = train_val_csv.dropna()

# Shape of the data: we could see that the number of rows remains the same as no null values were reported
data_to_use.shape

In [ ]:
X = data_to_use.drop(axis=1, columns=['attack_cat']) # X is a dataframe
X = X.drop(axis=1, columns=['label'])


y1 = data_to_use['attack_cat'].values # y is an array
y2 = data_to_use['label'].values

In [ ]:
import numpy as np

In [ ]:
unique_values = np.unique(y1)
print(unique_values)

In [ ]:
# Calculate Y2 ratio
def data_ratio(y2):
    '''
    Calculate Y2's ratio
    '''
    unique, count = np.unique(y2, return_counts=True)
    ratio = round(count[0]/count[1], 1)
    return f'{ratio}:1 ({count[0]}/{count[1]})'

In [ ]:
import pandas as pd
import numpy as np
import seaborn as sns
import matplotlib.pyplot as plt
plt.style.use('ggplot')
%matplotlib inline

# sklearn: data preprocessing
from sklearn.compose import ColumnTransformer
from sklearn.preprocessing import StandardScaler, LabelEncoder, OneHotEncoder, MinMaxScaler

# sklearn: train model
from sklearn.model_selection import train_test_split
from sklearn.model_selection import cross_val_score, cross_validate, StratifiedKFold
from sklearn.metrics import precision_recall_curve, precision_score, recall_score, f1_score, accuracy_score
from sklearn.metrics import roc_curve, auc, roc_auc_score, confusion_matrix, classification_report

# sklearn classifiers

from sklearn import tree
from sklearn.tree import DecisionTreeClassifier

from sklearn import datasets, ensemble, model_selection
from sklearn.ensemble import RandomForestClassifier

from sklearn.neural_network import MLPClassifier

In [ ]:
# print('The class ratio for the original data:', data_ratio(y1))
# plt.figure(figsize=(13,5))
# sns.countplot(y1,label="Sum")
# plt.show()

# # print('The class ratio for the original data:', data_ratio(y2))
# # sns.countplot(y2,label="Sum")
# # plt.show()

In [ ]:
# test_data = pd.read_csv('UNSW_NB15_testing.csv')
X_test = test_csv.drop(axis=1, columns=['attack_cat']) # X_test is a dataframe
X_test = X_test.drop(axis=1, columns=['label'])


y1_test = test_csv['attack_cat'].values # y is an array
y2_test = test_csv['label'].values

In [ ]:
X_val = val.drop(axis=1, columns=['attack_cat'])
X_val = X_val.drop(axis=1, columns=['label'])

y1_val = val['attack_cat'].values # y is an array
y2_val = val['label'].values

In [ ]:
X_train = X
y1_train = y1
y2_train = y2

In [ ]:
numerical_cols = X_train.select_dtypes(include=['int64', 'float64']).columns
categorical_cols = X_train.select_dtypes(include=['object', 'bool']).columns

In [ ]:
# define the transformation methods for the columns
t = [('ohe', OneHotEncoder(drop='first', handle_unknown='ignore'), categorical_cols),
    ('scale', MinMaxScaler(), numerical_cols)]

col_trans = ColumnTransformer(transformers=t)

# fit the transformation on training data
col_trans.fit(X_train)

In [ ]:
X_train_transform = col_trans.transform(X_train)

In [ ]:
X_test_transform = col_trans.transform(X_test)

In [ ]:
X_val_transform = col_trans.transform(X_val)

In [ ]:
target_trans = LabelEncoder()
target_trans.fit(y1_train)

In [ ]:
y1_train_transform = target_trans.transform(y1_train)
y1_test_transform = target_trans.transform(y1_test)
y1_val_transform = target_trans.transform(y1_val)

In [ ]:
target_trans = LabelEncoder()
target_trans.fit(y2_train)
y2_train_transform = target_trans.transform(y2_train)
y2_test_transform = target_trans.transform(y2_test)

In [ ]:
y2_train_transform

XGBoost Classifier


In [ ]:
from numpy import loadtxt
from xgboost import XGBClassifier
from matplotlib import pyplot

In [ ]:
model = XGBClassifier()
model.fit(X_train_transform, y1_train_transform)

In [ ]:
from xgboost import plot_importance
from matplotlib import pyplot

Decision Tree Classifer

In [ ]:
# Create Decision Tree classifer object
DTclf = DecisionTreeClassifier()

# Train Decision Tree Classifer
DTclf = DTclf.fit(X_train_transform, y1_train_transform)

y_pred_val = DTclf.predict(X_val_transform)

#Predict the response for test dataset
y_pred = DTclf.predict(X_test_transform)

In [ ]:
from sklearn.metrics import accuracy_score, precision_score, recall_score, roc_auc_score, average_precision_score, f1_score

In [ ]:
def calculate_classification_metrics(y_true, y_pred):
    # Calculate accuracy
    accuracy = accuracy_score(y_true, y_pred)

    # Calculate precision
    precision = precision_score(y_true, y_pred, average='weighted')

    # Calculate recall
    recall = recall_score(y_true, y_pred, average='weighted')


    # Calculate f1 score
    f1 = f1_score(y_true, y_pred, average='weighted')

    # Calculate macro F1 score
    macro_f1_score = f1_score(y_true, y_pred, average='macro')

    return accuracy, precision, recall, f1, macro_f1_score

In [ ]:
y_train_predict = DTclf.predict(X_train_transform)

In [ ]:
accuracy, precision, recall, f1, macro_f1 = calculate_classification_metrics(y1_train_transform, y_train_predict)
print("For Train Dataset: ")
print(f"Accuracy: {accuracy}")
print(f"Precision: {precision}")
print(f"Recall: {recall}")
# print(f"AUC: {auc}")
# print(f"PRC: {prc}")
print(f"f1: {f1}")
print(f"macro_f1: {macro_f1}")

print("-----------------")

In [ ]:
# Calculate accuracy
accuracy, precision, recall, f1, macro_f1 = calculate_classification_metrics(y1_val_transform, y_pred_val)
print("For Validation Dataset: ")
print(f"Accuracy: {accuracy}")
print(f"Precision: {precision}")
print(f"Recall: {recall}")
# print(f"AUC: {auc}")
# print(f"PRC: {prc}")
print(f"f1: {f1}")


print("-----------------")

In [ ]:
# Calculate accuracy
accuracy, precision, recall, f1, macro_f1 = calculate_classification_metrics(y1_test_transform, y_pred)
print("For Testing Dataset: ")
print(f"Accuracy: {accuracy}")
print(f"Precision: {precision}")
print(f"Recall: {recall}")
# print(f"AUC: {auc}")
# print(f"PRC: {prc}")
print(f"f1: {f1}")

print("-----------------")

**For Important Feature Selection**

In [ ]:
from numpy import sort

In [ ]:
thresholds = sort(model.feature_importances_)

In [ ]:
print(thresholds)

In [ ]:
from sklearn.feature_selection import SelectFromModel

In [ ]:
selection = SelectFromModel(model, threshold=0.0057178, prefit=True)
# print(X_train_transform[0])
select_X_train = selection.transform(X_train_transform)
# print(select_X_train[0])
select_X_test = selection.transform(X_test_transform)
print(select_X_train.shape, select_X_test.shape)

select_X_val = selection.transform(X_val_transform)

After feature selection

In [ ]:
# Create Decision Tree classifer object
selectDT = DecisionTreeClassifier()

# Train Decision Tree Classifer
selectDT = selectDT.fit(select_X_train, y1_train_transform)

#Predict the response for test dataset
y_pred = selectDT.predict(select_X_test)
y_pred_val = selectDT.predict(select_X_val)

In [ ]:
y_train_pred = selectDT.predict(select_X_train)

In [ ]:
accuracy, precision, recall, f1, macro_f1 = calculate_classification_metrics(y1_train_transform, y_train_pred)
print("For Training Dataset (After selecting important features): ")
print(f"Accuracy: {accuracy}")
print(f"Precision: {precision}")
print(f"Recall: {recall}")
print(f"f1: {f1}")

print("-----------------")

In [ ]:
accuracy, precision, recall, f1, macro_f1 = calculate_classification_metrics(y1_val_transform, y_pred_val)
print("For validation Dataset (After selecting important features): ")
print(f"Accuracy: {accuracy}")
print(f"Precision: {precision}")
print(f"Recall: {recall}")
print(f"f1: {f1}")

print("-----------------")

In [ ]:
accuracy, precision, recall, f1, macro_f1 = calculate_classification_metrics(y1_test_transform, y_pred)
print("For testing Dataset (After selecting important features): ")
print(f"Accuracy: {accuracy}")
print(f"Precision: {precision}")
print(f"Recall: {recall}")
print(f"f1: {f1}")


print("-----------------")